In [32]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import shapely

In [3]:
import sys
sys.path.append('..')

In [4]:
import utils

In [5]:
catalog_df = pd.read_csv('../data/era5/catalog.csv')

In [ ]:
catalog_df.describe()

In [7]:
def get_filepath(
    var:str,
    date:str,
    catalog_df = catalog_df,
):
    return catalog_df[
        (catalog_df['var'] == var) &
        (catalog_df['date'] == date)
    ]['filepath'].tolist()[0]

In [8]:
var = 'total_precipitation'
date = '1995-10-01'

tp_filepath_19951001 = get_filepath(var = 'total_precipitation', date = '1995-10-01')
tp_filepath_19951101 = get_filepath(var = 'total_precipitation', date = '1995-11-01')
tp_filepath_19951201 = get_filepath(var = 'total_precipitation', date = '1995-12-01')

In [9]:
tp_19951001 = utils.load_total_precipitation_nc_file(nc_filepath = tp_filepath_19951001)
tp_19951101 = utils.load_total_precipitation_nc_file(nc_filepath = tp_filepath_19951101)
tp_19951201 = utils.load_total_precipitation_nc_file(nc_filepath = tp_filepath_19951201)

In [ ]:
type(tp_19951201)

In [11]:
malawi_gdf = gpd.read_file('/gpfs/data1/cmongp1/sasirajann/download_era5/data/shapefiles/mwi_adm_nso_hotosm_20230405_shp/mwi_admbnda_adm0_nso_hotosm_20230405.shp')

In [ ]:
malawi_gdf.geometry.values

In [ ]:
malawi_gdf.crs

In [ ]:
malawi_gdf.envelope.values

In [15]:
def clip_data(
    data:xr.DataArray,
    shapes:list[shapely.Geometry],
    crs:str,
    drop:bool = True,
    all_touched:bool = True,
):
    return data.rio.clip(shapes, crs, drop=drop, all_touched=all_touched)

In [16]:
clipped_tp_19951001 = clip_data(
    data=tp_19951001,
    shapes=malawi_gdf.envelope.values, 
    crs=malawi_gdf.crs, 
    drop=True, 
    all_touched=True,
)

clipped_tp_19951101 = clip_data(
    data=tp_19951101,
    shapes=malawi_gdf.envelope.values, 
    crs=malawi_gdf.crs, 
    drop=True, 
    all_touched=True,
)

clipped_tp_19951201 = clip_data(
    data=tp_19951201,
    shapes=malawi_gdf.envelope.values, 
    crs=malawi_gdf.crs, 
    drop=True, 
    all_touched=True,
)

In [ ]:
clipped_tp_19951001.shape, clipped_tp_19951101.shape, clipped_tp_19951201.shape

In [28]:
clipped_tp_concat = xr.concat([
    clipped_tp_19951001,
    clipped_tp_19951101,
    clipped_tp_19951201,
], dim='valid_time')

In [ ]:
clipped_tp_concat.valid_time